## Import libraries

In [1]:
# !pip install geopandas
# !pip install unidecode
# !pip install awswrangler
# !pip install Levenshtein

In [ ]:
# In[2]:
import json
import pandas as pd
import numpy as np
import sys
import os
import os.path
import string
import random
import collections
#from sentence_transformers import SentenceTransformer, util
import warnings
warnings.filterwarnings('ignore')

#import helpers.py
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)

import helper


current = os.getcwd()
p = os.path.dirname(current)
parent = os.path.dirname(p)
sys.path.append(parent)

import helpers

In [ ]:
folder = "train_all_region123"

## I. Load dataset

In [4]:
df = pd.read_csv("foursquare/fold_train.csv")
df.columns

Index(['id', 'name', 'latitude', 'longitude', 'address', 'city', 'state',
       'zip', 'country', 'url', 'phone', 'categories', 'point_of_interest',
       'set'],
      dtype='object')

## II. Filtering

In [6]:
# filtering region
# US, CA, AU, GB, SG
countries = ['US', 'CA', 'AU', 'GB', 'SG']
#countries = ['US']
#countries = ['US']
df_test = df[df['country'].isin(countries)]
#df_test.columns
print("number of POIs in english countries:", len(df_test))

number of POIs in english countries: 314209


In [7]:
# TOO SLOW
# # Define a function to detect the language of a text string
# from langdetect import detect

# def detect_lang(text):
#     try:
#         lang = detect(text)
#     except:
#         lang = 'unknown'
#     return lang

# # Apply the function to each row in the dataframe
# df_test['language'] = df_test['name'].apply(detect_lang)

# # Filter out rows that are not in English
# df_test = df_test[df_test['language'] == 'en']
# print("number of POIs after filtering language:", len(df_test))

In [8]:
df_test.head(2)

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest,set
8,E_00007dcd2bb53f,TOGO'S Sandwiches,38.257797,-122.064599,"1380 Holiday Ln., Ste. B",Fairfield,CA,94534,US,https://locations.togos.com/ll/US/CA/Fairfield...,7074394747,Sandwich Places,P_aae7505da98d46,0.0
13,E_0000d9e584ed9f,Signature Properties Savannah,32.012582,-81.113156,100 Commercial Ct Ste C,Savannah,GA,31406,US,http://www.oursignatureproperties.com,9126292700,Real Estate Offices,P_af856e3abdcebc,1.0


In [9]:
float(len(df[df['country']=='US'])/len(df))

0.2153858582452591

In [10]:
#df_cat = pd.DataFrame(df_test['categories'].value_counts())
#df_cat = df_cat[(df_cat['categories']<=500) & (df_cat['categories']>300)]
#df_cat
#df_test[df_test['categories']=='General Travel']

In [11]:
# filtering categories (dining, lodging, experience)

# cats = ['Coffee Shops', 'Hotels', 'Pizza Places', 'Fast Food Restaurants', 'American Restaurants',
#        'Bars', 'Ethiopian Restaurants', 'Pools', 'Parks', 'Mexican Restaurants', 'Shopping Malls', 
#        'Restaurants', 'Cafés', 'Golf Courses', 'Food', 'Beaches', 'Lakes', 'Italian Restaurants', 'Sandwich Places',
#        'Chinese Restaurants', 'Sandwich Places, Fast Food Restaurants, Restaurants', 'Resorts',
#        'Ice Cream Shops', 'Other Great Outdoors', 'Bakeries', 'Campgrounds', 
#        'Donut Shops, Coffee Shops', 'Burger Joints', 'Sandwich Places, Fast Food Restaurants', 'Movie Theaters',
#        'Art Galleries', 'BBQ Joints', 'Liquor Stores', 'Seafood Restaurants', 'Fast Food Restaurants, Burger Joints',
#         'Breakfast Spots', 'Steakhouses', 'Diners']

# df_test = df_test[df_test['categories'].isin(cats)]
# print("number of POIs after filtering categories:", len(df_test))

In [12]:
# define attribute to keep
# name, coordinates, address, type

columns = ['id', 'name', 'latitude', 'longitude', 'address', 'city', 'state', 'country', 'zip', 'categories', 'point_of_interest']
df_test = df_test[columns]
df_test = df_test.reset_index(drop=True)

In [13]:
# remove missing values
df_test.isnull().sum()
df_test.dropna(inplace=True)

print("number of POIs after filtering missing values:", len(df_test))

number of POIs after filtering missing values: 205422


## III. Find matched/mismatched pairs from same POI

In [14]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [15]:
df_cnt = pd.DataFrame(df_test['point_of_interest'].value_counts())
df_cnt.head(2)

,point_of_interest
P_399ab9d64f2a2e,57
P_ce9291000a8f0b,46


In [16]:
def checkName(name1, name2):
    short_names = helpers.preprocessor(name1)
    long_names = helpers.preprocessor(name2)
    
    if len(short_names)>len(long_names):
        short_names, long_names = long_names, short_names
        
    # case 1: if concatenated words match 
    # (e.g. Caffe Bene vs. Caffebene)
    if "".join(short_names) == "".join(long_names):
        return True
     
    for w in short_names:
        if w in long_names:
            return True 
    return False
    

In [17]:
pairs = []


for cnt in range(2, 3): #2,3
 
    df_tmp = df_test[df_test['point_of_interest'].isin(list(df_cnt[df_cnt['point_of_interest']==cnt].index))]
    
    j = 0

    for poi, df in df_tmp.groupby("point_of_interest"):
        if j%1000 == 0:
            print(j)
        j+=1

        if len(df)!=cnt:
            print("error")

        name = df.iloc[0]['name']
        lat = df.iloc[0]['latitude']
        lon = df.iloc[0]['longitude']
        addr = df.iloc[0]['address']
        city = df.iloc[0]['city']
        state = df.iloc[0]['state']
        country = df.iloc[0]['country']
        zp = df.iloc[0]['zip']
        cat = df.iloc[0]['categories']
        poi = df.iloc[0]['point_of_interest']
        
        if not isEnglish(name):
            continue

        for i in range(1, cnt):
            name_i = df.iloc[i]['name']
            lat_i = df.iloc[i]['latitude']
            lon_i = df.iloc[i]['longitude']
            addr_i = df.iloc[i]['address']
            city_i = df.iloc[i]['city']
            state_i = df.iloc[i]['state']
            country_i = df.iloc[i]['country']
            zp_i = df.iloc[i]['zip']
            cat_i = df.iloc[i]['categories']
            poi_i = df.iloc[i]['point_of_interest']
            
            if not isEnglish(name_i):
                continue
            
            isNameMatch = checkName(name, name_i)
            isAddrMatch = helpers.checkAddress(addr, addr_i)
            isDistMatch = helpers.checkDistance(lat, lat_i, lon, lon_i)
            
            if isNameMatch & (isAddrMatch | isDistMatch):

                pairs.append([name, name_i, addr, addr_i, city, city_i, state, state_i, country, country_i,
                              zp, zp_i, cat, cat_i, lat, lat_i, lon, lon_i, poi, poi_i, 1])  
                    
                

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000


In [18]:
# save to csv
df_pairs = pd.DataFrame(pairs, columns=['name1', 'name2', 'addr1', 'addr2',  'city1', 'city2', 'state1', 'state2', 
                                        'country1', 'country2',
                                        'zip1', 'zip2', 'cat1', 'cat2', 'lat1', 'lat2', 'lon1', 'lon2', 'poi1', 'poi2', 'match'])

# split
df_pairs[df_pairs['match'] == 1].to_csv(folder + "/pairs_fs_match.csv")
# need to investiaget mismatched cases
#df_pairs[df_pairs['match'] == 0].to_csv(folder + "/pairs_fs_TBD.csv")

df_pairs['match'].value_counts()

1    38357
Name: match, dtype: int64

## IV. Find mismatch entries based on similarity 

In [20]:
df_zip = pd.DataFrame(df_test[['country', 'zip']].value_counts())
df_zip.head(2)
len(df_zip[df_zip[0]>=5])

8926

In [21]:
from sklearn.neighbors import KNeighborsRegressor

n_neighbors = 20

def add_neighbor_features(df):
    
    f = open("debug.txt", "a")
    dfs = []
    j = 0
    for sub, sub_df in df.groupby(["country", "zip"]):
        
        if len(sub_df)<5:
            continue
            
        if j%100==0:
            print(j)
            f.write("step:"+str(j)+"\n")
        j+=1
            
        sub_df = sub_df.reset_index(drop=True)
        
        knn = KNeighborsRegressor(n_neighbors=min(len(sub_df), n_neighbors), 
                                  metric='haversine', n_jobs=-1)
        knn.fit(sub_df[['latitude','longitude']], sub_df.index)
        dists, nears = knn.kneighbors(sub_df[['latitude','longitude']], return_distance=True)

        for i in range(min(len(sub_df), n_neighbors)):
            for c in columns:
                sub_df[f"near_{c}_{i}"] = sub_df[c].values[nears[:, i]]
                
        for i in range(min(len(sub_df), n_neighbors), n_neighbors):
            for c in columns:
                sub_df[f"near_{c}_{i}"] = np.nan


        dfs.append(sub_df)
    df = pd.concat(dfs).reset_index(drop=True)
    f.close()
    return df

df_test  =  add_neighbor_features(df_test)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900


In [22]:
import Levenshtein

def add_distance_features(df):
    
    # name, address, category, point_of_interest

    for i in range(n_neighbors):
    #    print(i)
        # for name 
        for c in ['name', 'address']:
            jaros = []
    
            for str1, str2 in df[[f"{c}", f"near_{c}_{i}"]].values.astype(str):
      #          print(str1, str2)
                str1, str2 = helper.preprocessor(str1), helper.preprocessor(str2)
      #          print(str1, str2)
                jaros.append(Levenshtein.jaro_winkler(str1, str2))
             #       lcss.append(LCS(str(str1), str(str2)))

            df[f"near_{c}_{i}_jaro"] = jaros
       
    return df

df_test = add_distance_features(df_test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [23]:
df_test.describe()

,latitude,longitude,near_latitude_0,near_longitude_0,near_latitude_1,near_longitude_1,near_latitude_2,near_longitude_2,near_latitude_3,near_longitude_3,...,near_name_15_jaro,near_address_15_jaro,near_name_16_jaro,near_address_16_jaro,near_name_17_jaro,near_address_17_jaro,near_name_18_jaro,near_address_18_jaro,near_name_19_jaro,near_address_19_jaro
count,168449.000000,168449.000000,168449.000000,168449.000000,168449.000000,168449.000000,168449.000000,168449.000000,168449.000000,168449.000000,...,168449.000000,168449.000000,168449.000000,168449.000000,168449.000000,168449.000000,168449.000000,168449.000000,168449.000000,168449.000000
mean,35.112198,-83.030310,35.112198,-83.030310,35.111136,-83.031579,35.110848,-83.031834,35.110873,-83.033076,...,0.461199,0.512087,0.458610,0.504276,0.454956,0.495925,0.452434,0.488017,0.449754,0.480624
std,13.789810,48.023386,13.789810,48.023386,13.791399,48.018940,13.791852,48.017617,13.791869,48.018368,...,0.167177,0.230873,0.170194,0.233238,0.172013,0.235442,0.174928,0.237615,0.176850,0.239484
min,-42.904136,-159.582896,-42.904136,-159.582896,-42.904136,-159.582896,-42.902428,-159.504532,-42.902493,-159.545422,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,33.576006,-100.775375,33.576006,-100.775375,33.571911,-100.777159,33.571328,-100.777590,33.571758,-100.784007,...,0.436819,0.461988,0.436508,0.461111,0.434127,0.459596,0.433333,0.458937,0.431746,0.457778
50%,38.595166,-85.667353,38.595166,-85.667353,38.595400,-85.668667,38.595388,-85.668790,38.594108,-85.668236,...,0.483287,0.533761,0.481481,0.529365,0.481481,0.523684,0.479720,0.518926,0.477778,0.513889
75%,40.963243,-77.042557,40.963243,-77.042557,40.965764,-77.044469,40.966600,-77.044855,40.965716,-77.044902,...,0.552699,0.611111,0.552778,0.607143,0.550654,0.603175,0.551117,0.600000,0.550000,0.597222
max,64.860402,153.613761,64.860402,153.613761,64.859939,153.613425,64.859939,153.613761,64.859939,153.613761,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
# look for similarity name, address & category but different POI 
# !!!region 123
pairs_mismatch = []
ids = set()
min_tol = 0.7
max_tol = 0.99 #IMPORTANT!!!
for index, row in df_test.iterrows():
    idx = row['id']
    name = row['name']
    lat = row['latitude']
    lon = row['longitude']
    addr = row['address']
    city = row['city']
    state = row['state']
    country = row['country']
    zp = row['zip']
    cat = row['categories']
    poi = row['point_of_interest']
    
    if not isEnglish(name):
        continue
    
    for i in range(n_neighbors):
        
        id_i = row[f'near_id_{i}']
        
        if type(id_i)==float:
            continue
            
        if idx == id_i:
            continue
        
        if tuple(sorted((idx, id_i))) in ids:
            continue
          
        # exclude the same poi
        if poi == row[f'near_point_of_interest_{i}']:
            continue
            
        if city != row[f'near_city_{i}']:
            continue
            
        if state != row[f'near_state_{i}']:
            continue
            
        if country != row[f'near_country_{i}']:
            continue 
            
        if not isEnglish(row[f'near_name_{i}']):
            continue
            
      #  if zp != row[f'near_zip_{i}']:
      #      continue
            
      #  if abs(lat-row[f'near_latitude_{i}'])>0.01:
      #      continue
            
     #   if abs(lon-row[f'near_longitude_{i}'])>0.01:
      #      continue
            
        # compute similarity scores
        jaro_name = row[f'near_name_{i}_jaro'] 
        jaro_addr = row[f'near_address_{i}_jaro'] 
        
    #    if jaro_name==1.0 and jaro_addr==1.0:
    #        continue
            
        if jaro_name<min_tol and jaro_addr<min_tol:
            continue
            
     #   if jaro_name>max_tol or jaro_name<=min_tol:
     #       continue
            
    #    if jaro_addr>max_tol or jaro_addr<=min_tol:
     #       continue
            
        ids.add(tuple(sorted((idx, id_i))))
        name_i = row[f'near_name_{i}']
        lat_i = row[f'near_latitude_{i}']
        lon_i = row[f'near_longitude_{i}']
        addr_i = row[f'near_address_{i}']
        city_i = row[f'near_city_{i}']
        state_i = row[f'near_state_{i}']
        country_i = row[f'near_country_{i}']
        zp_i = row[f'near_zip_{i}']
        cat_i = row[f'near_categories_{i}']
        poi_i = row[f'near_point_of_interest_{i}']
        
        pairs_mismatch.append([name, name_i, addr, addr_i, city, city_i, state, state_i, 
                               country, country_i,
                               zp, zp_i, cat, cat_i, lat, lat_i, lon, lon_i, poi, poi_i, jaro_name, jaro_addr, 
            #    row[f'near_name_{i}_gesh'], row[f'near_name_{i}_leven'], row[f'near_name_{i}_jaro'],  
            #    row[f'near_address_{i}_gesh'], row[f'near_address_{i}_leven'], row[f'near_address_{i}_jaro'],
            #    row[f'near_categories_{i}_gesh'], row[f'near_categories_{i}_leven'], row[f'near_categories_{i}_jaro'],
                               0])  
                
      

In [25]:
len(ids), len(pairs_mismatch), len(df_test)

(337325, 337325, 168449)

In [26]:
df_pairs_mismatch = pd.DataFrame(pairs_mismatch, columns=['name1', 'name2', 'addr1', 'addr2',  'city1', 'city2', 'state1', 'state2', 
                                                          'country1', 'country2', 
                                        'zip1', 'zip2', 'cat1', 'cat2', 'lat1', 'lat2', 'lon1', 'lon2', 'poi1', 'poi2', 'jaro_name', 'jaro_addr',
                                    #    'name_gesh', 'name_leven', 'name_jaro', 
                                    #    'address_gesh', 'address_leven', 'address_jaro',
                                    #    'cat_gesh', 'cat_leven', 'cat_jaro', 
                                                          'match'])

# split
df_pairs_mismatch.to_csv(folder + "/pairs_fs_mismatch.csv")

# optional

In [ ]:
# add cases with same address but different name which are mismatached
pairs_mismatch_same_addr = []

for index, row in df_test.iterrows():
    idx = row['id']
    name = row['name']
    lat = row['latitude']
    lon = row['longitude']
    addr = row['address']
    city = row['city']
    state = row['state']
    country = row['country']
    zp = row['zip']
    cat = row['categories']
    poi = row['point_of_interest']
    
    if not isEnglish(name):
        continue
    
    for i in range(1, n_neighbors):
        
        # exclude the same poi
        if poi == row[f'near_point_of_interest_{i}']:
            continue
            
        if city != row[f'near_city_{i}']:
            continue
            
        if state != row[f'near_state_{i}']:
            continue
            
        if country != row[f'near_country_{i}']:
            continue 
            
        if not isEnglish(row[f'near_name_{i}']):
            continue
            
      #  if zp != row[f'near_zip_{i}']:
      #      continue
            
      #  if abs(lat-row[f'near_latitude_{i}'])>0.01:
      #      continue
            
     #   if abs(lon-row[f'near_longitude_{i}'])>0.01:
      #      continue
            
        # compute similarity scores
        jaro_name = row[f'near_name_{i}_jaro'] 
        jaro_addr = row[f'near_address_{i}_jaro'] 
        
        if jaro_name==1.0 and jaro_addr==1.0:
            continue
            
        if jaro_name>max_tol or jaro_name<=min_tol:
            continue
            
        if jaro_addr!=1.0:
            continue
            
     
        id_i = row[f'near_id_{i}']
        
        if type(id_i)==float:
            continue
        
        if tuple(sorted((idx, id_i))) in ids:
            continue
            
        ids.add(tuple(sorted((idx, id_i))))
        name_i = row[f'near_name_{i}']
        lat_i = row[f'near_latitude_{i}']
        lon_i = row[f'near_longitude_{i}']
        addr_i = row[f'near_address_{i}']
        city_i = row[f'near_city_{i}']
        state_i = row[f'near_state_{i}']
        country_i = row[f'near_country_{i}']
        zp_i = row[f'near_zip_{i}']
        cat_i = row[f'near_categories_{i}']
        poi_i = row[f'near_point_of_interest_{i}']

        
        pairs_mismatch_same_addr.append([name, name_i, addr, addr_i, city, city_i, state, state_i, 
                               country, country_i,
                               zp, zp_i, cat, cat_i, lat, lat_i, lon, lon_i, poi, poi_i, jaro_name, jaro_addr, 
            #    row[f'near_name_{i}_gesh'], row[f'near_name_{i}_leven'], row[f'near_name_{i}_jaro'],  
            #    row[f'near_address_{i}_gesh'], row[f'near_address_{i}_leven'], row[f'near_address_{i}_jaro'],
            #    row[f'near_categories_{i}_gesh'], row[f'near_categories_{i}_leven'], row[f'near_categories_{i}_jaro'],
                               0])  
                

In [ ]:
df_pairs_mismatch_same_addr = pd.DataFrame(pairs_mismatch_same_addr, columns=['name1', 'name2', 'addr1', 'addr2',  'city1', 'city2', 'state1', 'state2', 
                                                          'country1', 'country2', 
                                        'zip1', 'zip2', 'cat1', 'cat2', 'lat1', 'lat2', 'lon1', 'lon2', 'poi1', 'poi2', 'jaro_name', 'jaro_addr',
                                    #    'name_gesh', 'name_leven', 'name_jaro', 
                                    #    'address_gesh', 'address_leven', 'address_jaro',
                                    #    'cat_gesh', 'cat_leven', 'cat_jaro', 
                                                          'match'])

# split
df_pairs_mismatch_same_addr.to_csv(folder + "/pairs_fs_mismatch_same_addr.csv")

In [44]:
len(df_pairs_mismatch_same_addr)/(6770+5242)

0.5636030636030636

In [ ]:
677